In [ ]:
# Importación de librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score


In [ ]:
# Preprocesamiento de datos para el analisis exploratorio de datos
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA

In [ ]:
# Cargar el dataset desde un archivo local
df = pd.read_csv("Mall_Customers.csv")

# Vista previa
print(df.head())

In [ ]:
# 2.  analisis exploratorio de datos
# Información básica del dataset
print("Información del dataset:")
print(df.info())

In [ ]:
# Resumen de estadisticas
print("\nResumen estadistico:")
print(df.describe())

In [ ]:
# Check for missing values
print("\nValores faltantes:")
print(df.isnull().sum())

In [ ]:
# Visualization of distributions
plt.figure(figsize=(15, 10))
# Histograma
numerical_cols = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 2, i)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribucion de {col}')

    plt.tight_layout()
    plt.show()

     # Mapa de calor de correlaciones
    plt.figure(figsize=(10, 8))
    correlation_matrix = df[numerical_cols].corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Mapa de calor de correlaciones')
    plt.show()

        # Caja bigote
    plt.figure(figsize=(15, 5))
    for i, col in enumerate(numerical_cols, 1):
        plt.subplot(1, 3, i)
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot de {col}')
    plt.tight_layout()
    plt.show()

In [ ]:
# 3. Preprocesamiento de datos
def preprocess_data(dataframe):
    # Select features for clustering
    features = ['Annual Income (k$)', 'Spending Score (1-100)']
    X = dataframe[features]

    # Standarizar las caracteristicas
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled, features

In [ ]:
# Preprocesar los datos
X_scaled, features = preprocess_data(df)

# 4. Método de elbow
def find_optimal_clusters(X_scaled):

    inertias = []
    silhouette_scores = []
    k_range = range(2, 11)

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))


    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(k_range, inertias, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Inertia')
    plt.title('Elbow Method')

    plt.subplot(1, 2, 2)
    plt.plot(k_range, silhouette_scores, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score')

    plt.tight_layout()
    plt.show()

    return inertias, silhouette_scores


In [ ]:
# Hallar el número óptimo de clusters
inertias, silhouette_scores = find_optimal_clusters(X_scaled)

# 5. Ejecutar k-clustering
def perform_clustering(X_scaled, n_clusters=5):
    # Perform K-Means Clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(X_scaled)

    # Compute clustering metrics
    silhouette = silhouette_score(X_scaled, cluster_labels)
    calinski = calinski_harabasz_score(X_scaled, cluster_labels)

    print(f"Clustering Metrics for {n_clusters} clusters:")
    print(f"Silhouette Score: {silhouette:.4f}")
    print(f"Calinski-Harabasz Index: {calinski:.4f}")

    return kmeans, cluster_labels

kmeans, cluster_labels = perform_clustering(X_scaled)

In [ ]:
def visualize_clusters(X_scaled, cluster_labels, features):
    # Create a DataFrame with scaled features and cluster labels
    cluster_df = pd.DataFrame(X_scaled, columns=features)
    cluster_df['Cluster'] = cluster_labels

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(cluster_df[features[0]],
                          cluster_df[features[1]],
                          c=cluster_df['Cluster'],
                          cmap='viridis')
    plt.xlabel(features[0])
    plt.ylabel(features[1])
    plt.title('Clusters of Mall Customers')
    plt.colorbar(scatter)
    plt.show()

    cluster_centers = kmeans.cluster_centers_
    print("\nCluster Centers:")
    center_df = pd.DataFrame(cluster_centers, columns=features)
    print(center_df)

    plt.figure(figsize=(10, 6))
    cluster_df['Cluster'].value_counts().plot(kind='bar')
    plt.title('Distribution of Customers in Each Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Number of Customers')
    plt.show()

In [ ]:
visualize_clusters(X_scaled, cluster_labels, features)

In [ ]:
def analyze_clusters(original_df, cluster_labels, features):
    clustered_df = original_df.copy()
    clustered_df['Cluster'] = cluster_labels

    cluster_analysis = clustered_df.groupby('Cluster')[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].mean()
    print("\nCluster Characteristics:")
    print(cluster_analysis)

    plt.figure(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    cluster_gender = pd.crosstab(clustered_df['Cluster'], clustered_df['Gender'], normalize='index')
    cluster_gender.plot(kind='bar', stacked=True, ax=plt.gca())
    plt.title('Gender Distribution in Clusters')
    plt.xlabel('Cluster')
    plt.ylabel('Proportion')

    plt.subplot(1, 2, 2)
    sns.boxplot(x='Cluster', y='Age', data=clustered_df)
    plt.title('Age Distribution in Clusters')

    plt.tight_layout()
    plt.show()

analyze_clusters(df, cluster_labels, features)

print("K-Means Clustering Analysis Complete!")
     